In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join('..', '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from tensorflow.keras.layers import LSTM, GRU
from cyberattacks_detection.models import RBFNN, ELM, ModelWrapper, min_max_scale, reverse_min_max_scale, MinMaxScalerLayer, create_rnn_input, create_rnn, predict_recursion, create_recurrent_and_mlp_model
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import re
from itertools import product
from tqdm import tqdm

c:\Users\zgore\anaconda3\envs\cyber-attk-p39\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:103: FutureWarning: Model's `predict` method contains invalid parameters: {'X'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
c:\Users\zgore\anaconda3\envs\cyber-attk-p39\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:134: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [3]:
import mlflow
from mlflow.models import infer_signature

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Rescaling, GRU
# from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Lambda
import tensorflow as tf

In [5]:
pd.set_option('display.max_colwidth', None)

# Przygotowanie danych

In [6]:
data_path = "../../data/four_tanks/"

In [7]:
# # dane bez zakłóceń
# df_train = pd.read_csv(data_path+"result_ol_without_noise_uczące_v3.csv", sep=';', index_col=0)
# df_val = pd.read_csv(data_path+"result_ol_without_noise_walidacyjne_v3.csv", sep=';', index_col=0)
# df_test = pd.read_csv(data_path+"result_ol_without_noise_testowe_v3.csv", sep=';', index_col=0)
# dataset_name = "bez zakłóceń"

In [8]:
# dane z zakłóceniami
df_train = pd.read_csv(data_path+"result_ol_with_noise_uczące_v5.csv", sep=';', index_col=0)
df_val = pd.read_csv(data_path+"result_ol_with_noise_walidacyjne_v5.csv", sep=';', index_col=0)
df_test = pd.read_csv(data_path+"result_ol_with_noise_testowe_v5.csv", sep=';', index_col=0)
dataset_name = "z zakłóceniami"

In [9]:
df_train.head()

,q_A [cm^3/s],q_B [cm^3/s],q_d1 [cm^3/s],q_d2 [cm^3/s],q_d3 [cm^3/s],q_d4 [cm^3/s],x1 [cm],x2 [cm],x3 [cm],x4 [cm]
0,452.78,555.56,0.2646,0.2390,0.3064,0.2126,65.000000,66.000000,65.000000,66.000000
1,452.78,555.56,0.0600,0.0853,-0.1379,0.0666,65.000000,66.000000,65.000000,66.000000
2,452.78,555.56,0.1468,-0.0172,0.0172,-0.0115,65.000000,66.000000,65.000000,66.000000
3,452.78,555.56,0.3361,0.0377,-0.0206,-0.1637,65.000000,66.000000,65.000000,66.000000
4,452.78,555.56,0.2801,-0.1816,0.2048,-0.0656,65.320426,65.975004,65.017598,65.828939


In [10]:
def transform_column_name(df):
    df.columns = [col[0] for col in df.columns.str.split()]
    return df

In [11]:
df_train = transform_column_name(df_train)
df_val = transform_column_name(df_val)
df_test = transform_column_name(df_test)

In [12]:
qa_max = round(3260000/3600, 2)
qa_min=0
qb_max = round(4000000/3600)
qb_min=0
x_max = [136,
         136,
         130,
         130]
x_min = [20,
         20,
         20,
         20]

In [13]:
def add_delay(df, col, steps=1):    
    df[col+f'(k-{steps})'] = df[col].shift(steps)
    return df

In [14]:
def add_delay_to_all_cols(df_train, df_val, df_test):
    cols_delay = ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']
    for col in cols_delay:
        for step in range(1, 5):
            df_train = add_delay(df_train, col, step)
            df_val = add_delay(df_val, col, step)
            df_test = add_delay(df_test, col, step)
    return df_train, df_val, df_test

In [15]:
# df_train, df_val, df_test = add_delay_to_all_cols(df_train, df_val, df_test)

In [16]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)
df_test.dropna(inplace=True)

In [17]:
print(f"Wymiar danych uczących: {df_train.shape}")
print(f"Wymiar danych walidacyjnych: {df_val.shape}")
print(f"Wymiar danych testowych: {df_test.shape}")

Wymiar danych uczących: (2999, 10)
Wymiar danych walidacyjnych: (999, 10)
Wymiar danych testowych: (999, 10)


# Modelowanie

In [18]:
mlflow.set_tracking_uri("http://localhost:5000")

In [19]:
model_type_dict = {'statespace': 'w przestrzeni stanu', 'io': 'wejście-wyjście'}
model_class_list = ["LR", "ELM", "RBF", "LSTM", 'GRU', 'LSTM-MLP', 'GRU-MLP']

In [61]:
def normalized_x_error(RMSE, min_val_y, max_val_y):
    return RMSE/(max_val_y - min_val_y)

In [62]:
def prepare_data(df_train, df_val, df_test, features, y_name, time_steps, min_val_y, max_val_y):
    col_idx = df_train[features].columns.get_loc(y_name)
    X_train, y_train = create_rnn_input(df_train[features], col_idx, time_steps)
    y_train_sc = min_max_scale(y_train, min_val_y, max_val_y)
    
    X_val, y_val = create_rnn_input(df_val[features], col_idx, time_steps)
    y_val_sc = min_max_scale(y_val, min_val_y, max_val_y)
    
    X_test, y_test = create_rnn_input(df_test[features], col_idx, time_steps)
    
    return X_train, y_train, y_train_sc, X_val, y_val, y_val_sc, X_test, y_test

In [63]:
def get_model(model_class_str, **kwargs):
    if model_class_str in ("LSTM-MLP", "GRU-MLP"):
        return create_recurrent_and_mlp_model(**kwargs)
    else:
        return create_rnn(**kwargs)

In [64]:
def evaluate_model(model, df_train, df_val, df_test, X_train, y_train, X_val, y_val, X_test, y_test, 
                   y_name, features, time_steps, num_features, min_val_y, max_val_y, pred_mode):
    metrics_not_rec, metrics_norm_not_rec = {}, {}
    metrics_rec, metrics_norm_rec = {}, {}

    pred_mode_tuple = ("both", "not_recursion", "recursion")
    if pred_mode not in pred_mode_tuple:
        raise ValueError(f"Invalid pred_mode. Expected one of {pred_mode_tuple}")

    if pred_mode in ("both", "not_recursion"):
        y_train_pred = reverse_min_max_scale(model.predict(X_train, verbose=0), min_val_y, max_val_y)
        y_val_pred = reverse_min_max_scale(model.predict(X_val, verbose=0), min_val_y, max_val_y)
        y_test_pred = reverse_min_max_scale(model.predict(X_test, verbose=0), min_val_y, max_val_y)

        metrics_not_rec = {
            "RMSE_train": round(metrics.root_mean_squared_error(y_train, y_train_pred), 4),
            "RMSE_val": round(metrics.root_mean_squared_error(y_val, y_val_pred), 4),
            "RMSE_test": round(metrics.root_mean_squared_error(y_test, y_test_pred), 4)
        }

        metrics_norm_not_rec = {
            f"NRMSE_{key}": round(normalized_x_error(val, min_val_y, max_val_y), 4)
            for key, val in metrics_not_rec.items()
        }

    if pred_mode in ("both", "recursion"):
        y_val_pred = predict_recursion(df_val, model, features, y_name, num_features, time_steps, min_val_y, max_val_y)
        RMSE_val = round(metrics.root_mean_squared_error(y_val, y_val_pred[time_steps:]), 4)

        if (RMSE_val > 6) or (y_name in ('x3', 'x4') and RMSE_val > 1.2):
            metrics_rec = {"RMSE_val_recursion2": RMSE_val}
            metrics_norm_rec = {"NRMSE_val_recursion2": round(normalized_x_error(RMSE_val, min_val_y, max_val_y), 4)}
        else:
            y_train_pred = predict_recursion(df_train, model, features, y_name, num_features, time_steps, min_val_y, max_val_y)
            y_test_pred = predict_recursion(df_test, model, features, y_name, num_features, time_steps, min_val_y, max_val_y)
            RMSE_train = round(metrics.root_mean_squared_error(y_train, y_train_pred[time_steps:]), 4)
            RMSE_test = round(metrics.root_mean_squared_error(y_test, y_test_pred[time_steps:]), 4)

            metrics_rec = {
                "RMSE_train_recursion": RMSE_train,
                "RMSE_val_recursion": RMSE_val,
                "RMSE_test_recursion": RMSE_test
            }
            metrics_norm_rec = {
                "NRMSE_train_recursion": round(normalized_x_error(RMSE_train, min_val_y, max_val_y), 4),
                "NRMSE_val_recursion": round(normalized_x_error(RMSE_val, min_val_y, max_val_y), 4),
                "NRMSE_test_recursion": round(normalized_x_error(RMSE_test, min_val_y, max_val_y), 4)
            }

        if y_name == 'x1':
            metrics_rec.update({k + "2": v for k, v in metrics_rec.items()})
            metrics_norm_rec.update({k + "2": v for k, v in metrics_norm_rec.items()})

    return metrics_not_rec | metrics_norm_not_rec | metrics_rec | metrics_norm_rec

In [65]:
def log_metrics_to_mlflow(params, metrics, model, model_class_str, model_type, time_steps, y_name, X_train, y_train_pred):
    with mlflow.start_run():
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        signature = infer_signature(X_train, y_train_pred)
        mlflow.keras.log_model(
            model,
            artifact_path=model_class_str,
            signature=signature,
            registered_model_name=f"{model_class_str}_{model_type}_{time_steps}_{y_name}"
        )

In [66]:
def train_and_evaluate_model(df_train,
                                df_val,
                                df_test,
                                model_class_str,
                                y_name,
                                features,
                                model_type,
                                time_steps,
                                min_vals,
                                max_vals,
                                patience,
                                optimizer,
                                loss,
                                epochs,
                                batch_size,
                                pred_mode='both',
                                **kwargs):

    idx = int(re.findall(r'x(\d+)', y_name)[0]) - 1
    # mlflow.set_experiment(f"four_tanks_{y_name.replace('x', 'h')}_v2")
    mlflow.set_experiment(f"test_{y_name.replace('x', 'h')}_v2")

    X_train, y_train, y_train_sc, X_val, y_val, y_val_sc, X_test, y_test = prepare_data(
        df_train, df_val, df_test, features, y_name, time_steps, x_min[idx], x_max[idx]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    model = get_model(model_class_str, num_features=len(features), time_steps=time_steps,
                       min_vals=min_vals, max_vals=max_vals, **kwargs)
    model.compile(optimizer=optimizer, loss=loss)

    history = model.fit(X_train, y_train_sc, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val_sc), callbacks=[callback])

    epochs_real = len(history.epoch)
    y_train_pred_sc = model.predict(X_train, verbose=0)
    y_train_pred_not_rec = reverse_min_max_scale(y_train_pred_sc, x_min[idx], x_max[idx])

    base_params = {
        "model_class": model_class_str,
        "model_type": model_type_dict[model_type],
        "order_dyn": time_steps,
        "y_name": y_name + '(k)',
        "features": features,
        "iterations": epochs,
        "iterations_real": epochs_real,
        "patience": patience,
        "optimizer": optimizer,
        "loss": loss,
        "batch_size": batch_size
    }

    metrics = evaluate_model(
        model, df_train, df_val, df_test,
        X_train, y_train, X_val, y_val, X_test, y_test,
        y_name, features, time_steps, len(features), x_min[idx], x_max[idx], pred_mode
    )

    log_metrics_to_mlflow(base_params | kwargs, metrics, model, model_class_str, model_type, time_steps, y_name, X_train, y_train_pred_not_rec)


In [67]:
def train_and_evaluate_4models(df_train, df_val, df_test,
                          model_class_str, y_names, features_list,
                          model_type, time_steps, min_vals_list, max_vals_list,
                          patience, optimizer, loss, epochs, batch_size,
                          pred_mode='both', **kwargs):
    for y_name, features, min_vals, max_vals in zip(y_names, features_list, min_vals_list, max_vals_list):
        train_and_evaluate_model(df_train, df_val, df_test,
                                    model_class_str, y_name, features,
                                    model_type, time_steps,
                                    min_vals, max_vals,
                                    patience, optimizer, loss,
                                    epochs, batch_size,
                                    pred_mode=pred_mode,
                                    **kwargs)

## LSTM

In [38]:
create_model_fn = create_rnn
model_class_str = model_class_list[3]
print(model_class_str)
recurrent_layer = LSTM

LSTM


In [ ]:
patience = 30
optimizer = 'adam'
loss = 'mse'
epochs = 400 # 10 # 300
batch_size = 32

In [ ]:
features_list = [['q_A', 'q_B', 'x1'],
                ['q_A', 'q_B', 'x2'],
                ['q_A', 'q_B', 'x3'],
                ['q_A', 'q_B', 'x4']]
y_names = ['x1', 'x2', 'x3', 'x4']
model_type = 'io'
min_vals_list = [np.array([qa_min, qb_min, x_min[0]]),
                np.array([qa_min, qb_min, x_min[1]]),
                np.array([qa_min, qb_min, x_min[2]]),
                np.array([qa_min, qb_min, x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0]]),
                np.array([qa_max, qb_max, x_max[1]]),
                np.array([qa_max, qb_max, x_max[2]]),
                np.array([qa_max, qb_max, x_max[3]])]

num_hidden_neurons_list = [8, 16, 24, 32, 48, 64]
num_hidden_layers_list = [1, 2, 3]
time_steps_list = [1, 2, 3, 4]

for num_hidden_neurons, num_hidden_layers, time_steps in tqdm(product(num_hidden_neurons_list, num_hidden_layers_list, time_steps_list),
                                                              total=len(num_hidden_neurons_list) * len(num_hidden_layers_list) * len(time_steps_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_hidden_layers=num_hidden_layers,
                            units_per_layer=num_hidden_neurons)

In [ ]:
features_list = [['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']]
y_names = ['x1', 'x2', 'x3', 'x4']
model_type = 'statespace'
min_vals_list = [np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]])]

num_hidden_neurons_list = [8, 16, 24, 32, 48, 64]
num_hidden_layers_list = [1, 2, 3]
time_steps = 1

for num_hidden_neurons, num_hidden_layers in tqdm(product(num_hidden_neurons_list, num_hidden_layers_list),
                                                              total=len(num_hidden_neurons_list) * len(num_hidden_layers_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            num_hidden_layers,
                            num_hidden_neurons,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_hidden_layers=num_hidden_layers,
                            units_per_layer=num_hidden_neurons)

## GRU

In [ ]:
model_class_str = model_class_list[4]
print(model_class_str)
recurrent_layer = GRU

GRU


In [ ]:
patience = 30
optimizer = 'adam'
loss = 'mse'
epochs = 400 # 10 # 300
batch_size = 32

In [ ]:
features_list = [['q_A', 'q_B', 'x1'],
                ['q_A', 'q_B', 'x2'],
                ['q_A', 'q_B', 'x3'],
                ['q_A', 'q_B', 'x4']]
y_names = ['x1', 'x2', 'x3', 'x4']
model_type = 'io'
min_vals_list = [np.array([qa_min, qb_min, x_min[0]]),
                np.array([qa_min, qb_min, x_min[1]]),
                np.array([qa_min, qb_min, x_min[2]]),
                np.array([qa_min, qb_min, x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0]]),
                np.array([qa_max, qb_max, x_max[1]]),
                np.array([qa_max, qb_max, x_max[2]]),
                np.array([qa_max, qb_max, x_max[3]])]

num_hidden_neurons_list = [8, 16, 24, 32, 48, 64]
num_hidden_layers_list = [1, 2, 3]
time_steps_list = [1, 2, 3, 4]

for num_hidden_neurons, num_hidden_layers, time_steps in tqdm(product(num_hidden_neurons_list, num_hidden_layers_list, time_steps_list),
                                                              total=len(num_hidden_neurons_list) * len(num_hidden_layers_list) * len(time_steps_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_hidden_layers=num_hidden_layers,
                            units_per_layer=num_hidden_neurons)

In [ ]:
features_list = [['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']]
y_names = ['x1', 'x2', 
           'x3', 'x4']
model_type = 'statespace'
min_vals_list = [np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]])]

num_hidden_neurons_list = [8, 16, 24, 32, 48, 64]
num_hidden_layers_list = [1, 2, 3]
time_steps_list = [1, 2, 3, 4]

for num_hidden_neurons, num_hidden_layers, time_steps in tqdm(product(num_hidden_neurons_list, num_hidden_layers_list, time_steps_list),
                                                              total=len(num_hidden_neurons_list) * len(num_hidden_layers_list) * len(time_steps_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_hidden_layers=num_hidden_layers,
                            units_per_layer=num_hidden_neurons)

# LSTM-MLP

In [68]:
model_class_str = model_class_list[5]
print(model_class_str)
recurrent_layer = LSTM

LSTM-MLP


In [ ]:
patience = 30
optimizer = 'adam'
loss = 'mse'
epochs = 400 # 10 # 300
batch_size = 32

In [ ]:
features_list = [['q_A', 'q_B', 'x1'],
                ['q_A', 'q_B', 'x2'],
                ['q_A', 'q_B', 'x3'],
                ['q_A', 'q_B', 'x4']]
y_names = ['x1', 'x2', 'x3', 'x4']
model_type = 'io'
min_vals_list = [np.array([qa_min, qb_min, x_min[0]]),
                np.array([qa_min, qb_min, x_min[1]]),
                np.array([qa_min, qb_min, x_min[2]]),
                np.array([qa_min, qb_min, x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0]]),
                np.array([qa_max, qb_max, x_max[1]]),
                np.array([qa_max, qb_max, x_max[2]]),
                np.array([qa_max, qb_max, x_max[3]])]

activation_mlp_list=['relu', 'tanh']
num_mlp_hidden_layers_list=[1, 2]
time_steps_list=[1]
recurrent_units_per_layer_list=[8, 16, 24, 32, 48, 64]
mlp_units_per_layer_list=[4, 8, 16, 24, 32, 48]

for activation_mlp, num_mlp_hidden_layers, time_steps, recurrent_units_per_layer, mlp_units_per_layer in tqdm(product(activation_mlp_list,
                                                                                                               num_mlp_hidden_layers_list,
                                                                                                              time_steps_list,
                                                                                                               recurrent_units_per_layer_list,
                                                                                                               mlp_units_per_layer_list),
                                                              total=len(recurrent_units_per_layer_list) * len(mlp_units_per_layer_list) * len(num_mlp_hidden_layers_list) * len(time_steps_list) * len(activation_mlp_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_mlp_hidden_layers=num_mlp_hidden_layers,
                            recurrent_units_per_layer=recurrent_units_per_layer,
                            mlp_units_per_layer=mlp_units_per_layer,
                            activation_mlp=activation_mlp)

In [ ]:
features_list = [['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']]
y_names = ['x1', 'x2', 
           'x3', 'x4']
model_type = 'statespace'
min_vals_list = [np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]])]

activation_mlp_list=['relu', 'tanh']
num_mlp_hidden_layers_list=[1, 2]
time_steps_list=[1]
recurrent_units_per_layer_list=[8, 16, 24, 32, 48, 64]
mlp_units_per_layer_list=[4, 8, 16, 24, 32, 48]

for activation_mlp, num_mlp_hidden_layers, time_steps, recurrent_units_per_layer, mlp_units_per_layer in tqdm(product(activation_mlp_list,
                                                                                                               num_mlp_hidden_layers_list,
                                                                                                              time_steps_list,
                                                                                                               recurrent_units_per_layer_list,
                                                                                                               mlp_units_per_layer_list),
                                                              total=len(recurrent_units_per_layer_list) * len(mlp_units_per_layer_list) * len(num_mlp_hidden_layers_list) * len(time_steps_list) * len(activation_mlp_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_mlp_hidden_layers=num_mlp_hidden_layers,
                            recurrent_units_per_layer=recurrent_units_per_layer,
                            mlp_units_per_layer=mlp_units_per_layer,
                            activation_mlp=activation_mlp)

# GRU-MLP

In [ ]:
model_class_str = model_class_list[6]
print(model_class_str)

recurrent_layer = GRU

In [ ]:
patience = 30
optimizer = 'adam'
loss = 'mse'
epochs = 400 # 10 # 300
batch_size = 32

In [ ]:
features_list = [['q_A', 'q_B', 'x1'],
                ['q_A', 'q_B', 'x2'],
                ['q_A', 'q_B', 'x3'],
                ['q_A', 'q_B', 'x4']]
y_names = ['x1', 'x2', 'x3', 'x4']
model_type = 'io'
min_vals_list = [np.array([qa_min, qb_min, x_min[0]]),
                np.array([qa_min, qb_min, x_min[1]]),
                np.array([qa_min, qb_min, x_min[2]]),
                np.array([qa_min, qb_min, x_min[3]])]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0]]),
                np.array([qa_max, qb_max, x_max[1]]),
                np.array([qa_max, qb_max, x_max[2]]),
                np.array([qa_max, qb_max, x_max[3]])]

activation_mlp_list=['relu', 'tanh']
num_mlp_hidden_layers_list=[1, 2]
time_steps_list=[1]
recurrent_units_per_layer_list=[8, 16, 24, 32, 48, 64]
mlp_units_per_layer_list=[4, 8, 16, 24, 32, 48]

for activation_mlp, num_mlp_hidden_layers, time_steps, recurrent_units_per_layer, mlp_units_per_layer in tqdm(product(activation_mlp_list,
                                                                                                               num_mlp_hidden_layers_list,
                                                                                                              time_steps_list,
                                                                                                               recurrent_units_per_layer_list,
                                                                                                               mlp_units_per_layer_list),
                                                              total=len(recurrent_units_per_layer_list) * len(mlp_units_per_layer_list) * len(num_mlp_hidden_layers_list) * len(time_steps_list) * len(activation_mlp_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_mlp_hidden_layers=num_mlp_hidden_layers,
                            recurrent_units_per_layer=recurrent_units_per_layer,
                            mlp_units_per_layer=mlp_units_per_layer,
                            activation_mlp=activation_mlp)

In [ ]:
features_list = [['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4'],
                 ['q_A', 'q_B', 'x1', 'x2', 'x3', 'x4']
                 ]
y_names = ['x1', 'x2'
           , 'x3', 'x4'
           ]
model_type = 'statespace'
min_vals_list = [np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]]),
                 np.array([qa_min, qb_min, x_min[0], x_min[1], x_min[2], x_min[3]])
                 ]
max_vals_lits = [np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]]),
                 np.array([qa_max, qb_max, x_max[0], x_max[1], x_max[2], x_max[3]])
                 ]
                 

activation_mlp_list=['relu', 'tanh']
num_mlp_hidden_layers_list=[1, 2]
time_steps_list=[1]
recurrent_units_per_layer_list=[8, 16, 24, 32, 48, 64]
mlp_units_per_layer_list=[4, 8, 16, 24, 32, 48]

for activation_mlp, num_mlp_hidden_layers, time_steps, recurrent_units_per_layer, mlp_units_per_layer in tqdm(product(activation_mlp_list,
                                                                                                               num_mlp_hidden_layers_list,
                                                                                                              time_steps_list,
                                                                                                               recurrent_units_per_layer_list,
                                                                                                               mlp_units_per_layer_list),
                                                              total=len(recurrent_units_per_layer_list) * len(mlp_units_per_layer_list) * len(num_mlp_hidden_layers_list) * len(time_steps_list) * len(activation_mlp_list),
                                                                desc="Training Progress"):
    train_and_evaluate_4models(df_train,
                            df_val,
                            df_test,
                            model_class_str,
                            y_names,
                            features_list,
                            model_type,
                            time_steps,
                            min_vals_list,
                            max_vals_lits,
                            patience,
                            optimizer,
                            loss,
                            epochs,
                            batch_size,
                            pred_mode='both',
                            recurrent_layer=recurrent_layer,
                            num_mlp_hidden_layers=num_mlp_hidden_layers,
                            recurrent_units_per_layer=recurrent_units_per_layer,
                            mlp_units_per_layer=mlp_units_per_layer,
                            activation_mlp=activation_mlp)